In [1]:
import pandas as pd

data = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2, 3, 3],
    'item_id': [101, 102, 103, 101, 104, 102, 103],
    'target': [1, 0, 1, 0, 1, 1, 0],  # Целевая метка, например, факт покупки
    'feature_1': [0.1, 0.2, 0.3, 0.1, 0.4, 0.2, 0.3],
    'feature_2': [10, 20, 30, 10, 40, 20, 30]
})

# Фичи, которые мы будем использовать для обучения
X = data[['user_id', 'item_id', 'feature_1', 'feature_2']]
y = data['target']


# Создание группового идентификатора для ранжирования (группировка по пользователям)
group_id = data['user_id']


In [2]:
from catboost import CatBoostRanker, Pool

# Определяем пул данных для обучения
train_pool = Pool(data=X, label=y, group_id=group_id)

# Настройка и обучение модели
ranker = CatBoostRanker(
    iterations=100,        # Количество итераций
    learning_rate=0.1,     # Скорость обучения
    depth=6,               # Глубина деревьев
    random_seed=42,
    verbose=10             # Показывать прогресс обучения
)

# Обучение модели
ranker.fit(train_pool)


0:	total: 65.7ms	remaining: 6.51s
10:	total: 72.5ms	remaining: 586ms
20:	total: 77.7ms	remaining: 292ms
30:	total: 83ms	remaining: 185ms
40:	total: 88.5ms	remaining: 127ms
50:	total: 93.1ms	remaining: 89.5ms
60:	total: 96.9ms	remaining: 61.9ms
70:	total: 101ms	remaining: 41.1ms
80:	total: 105ms	remaining: 24.6ms
90:	total: 109ms	remaining: 10.7ms
99:	total: 113ms	remaining: 0us


In [3]:
# Предсказание ранжирования
test_pool = Pool(data=X, group_id=group_id)
predictions = ranker.predict(test_pool)

# Добавляем предсказания в таблицу данных и сортируем
data['prediction'] = predictions
ranked_data = data.sort_values(by=['user_id', 'prediction'], ascending=[True, False])

print(ranked_data[['user_id', 'item_id', 'prediction']])


   user_id  item_id  prediction
2        1      103    8.341870
0        1      101    7.777660
1        1      102  -11.297562
4        2      104    9.109395
3        2      101  -11.009011
5        3      102    8.422453
6        3      103  -11.344805
